<a href="https://colab.research.google.com/github/Marieta3/sentiment-analysis-russian/blob/master/sentiment_analiza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Analiza sentimenta tvitova na ruskom jeziku


##Importovanje potrebnih biblioteka

In [0]:
import tensorflow as tf
import tflearn
import pandas as pd
from nltk.stem.snowball import RussianStemmer
from nltk.tokenize import TweetTokenizer
import re
from collections import Counter
from datetime import datetime
import numpy as np
from tflearn.data_utils import to_categorical

##Konstante

In [0]:
VOCAB_SIZE = 5000
MANJI_DEO = False
TEST_SIZE = 0.3

##Učitavanje podataka

In [0]:
col = 3 #kolona koja sadrzi tvit, ostale kolone nisu potrebne
rows=100000
if(MANJI_DEO):
  positive = pd.read_csv(
      'https://raw.githubusercontent.com/Marieta3/sentiment-analysis-russian/master/positive.csv', header=None, delimiter=';')[:rows][col];
  negative = pd.read_csv(
      'https://raw.githubusercontent.com/Marieta3/sentiment-analysis-russian/master/negative.csv', header=None, delimiter=';')[:rows][col];
else:
  positive = pd.read_csv(
      'https://raw.githubusercontent.com/Marieta3/sentiment-analysis-russian/master/positive.csv', header=None, delimiter=';')[[col]];
  negative = pd.read_csv(
      'https://raw.githubusercontent.com/Marieta3/sentiment-analysis-russian/master/negative.csv', header=None, delimiter=';')[[col]];  

##Testiranje učitanih podataka


In [0]:
print('Pozitivnih tvitova: ' +str(len(positive)))
print('Negativnih tvitova: ' +str(len(negative)))


Pozitivnih tvitova: 114911
Negativnih tvitova: 111923


##Izvlačenje osnovnog oblika iz tokena

In [0]:
stemmer = RussianStemmer()
stem_cache={}  #smanjuje vreme izvršavanja, ako se reč već pojavljivala, samo je uzmemo iz liste
def get_stem(token):
  stem =stem_cache.get(token, None)
  if stem:
    return stem
  
  token=re.sub('[^а-яА-Я ]', '', token)  #samo slova
  stem=stemmer.stem(token)
  stem_cache[token]=stem
  return stem
get_stem('!')

''

##Tokenizacija

In [0]:
recnik = Counter()
tokenizer = TweetTokenizer()


def popuni_recnik(tweets):
  for t in tweets.values.tolist():
    #print(t[0])
    tweet=t[0]
    
    tokens=tokenizer.tokenize(tweet)
    for token in tokens:
      stem=get_stem(token)
      if stem!='':
        recnik[stem]+=1
    
start_time=datetime.now()  
popuni_recnik(positive)
popuni_recnik(negative)
end_time=datetime.now()

In [0]:
print('Потребно време за попуњавање речника: ', end_time-start_time)
print("Укупан број различитих речи: ", len(recnik))


Потребно време за попуњавање речника:  0:01:17.361479
Укупан број различитих речи:  91779
['не', 'я', 'и', 'в', 'на', 'а', 'что', 'так', 'с', 'эт']


In [0]:
vocab = sorted(recnik, key=recnik.get, reverse=True)[:VOCAB_SIZE]  #сортирано по броју понављања опадајуће, првих 5000

In [0]:
print(vocab[:10])
print(vocab[1000:1020])
print(vocab[-10:])
print("Дужина скраћеног речника: ",len(vocab))

['не', 'я', 'и', 'в', 'на', 'а', 'что', 'так', 'с', 'эт']
['хз', 'причин', 'пропа', 'биолог', 'придт', 'выбор', 'ух', 'глуп', 'помо', 'шерлок', 'ссылк', 'горяч', 'устро', 'прода', 'назва', 'солнц', 'молодежк', 'тест', 'руга', 'эмоц']
['ае', 'зураг', 'юма', 'обучен', 'пьем', 'гал', 'гденибуд', 'нац', 'частн', 'сброс']
Дужина скраћеног речника:  5000


In [0]:
idx=5000-3
print("Reč: ", vocab[idx], " se pojavila ", recnik.get(vocab[idx]), " puta")




Reč:  нац  se pojavila  33  puta


In [0]:
token_to_idx = {vocab[i] : i for i in range(VOCAB_SIZE)}
print(token_to_idx.get('', None))
print(type(token_to_idx))

0
<class 'dict'>


##Pretvaranje tvita u vektor

*   vektor je duzine broja reci u recniku (5000)
*   ukoliko tvit sadrzi rec koja se nalazi u recniku, na odgovarajucem mestu u vektoru se nalazi jedinica; u suprotnom nula


In [0]:
def tweet_to_vector(tweet, print_unknown=False):
  vector=np.zeros(VOCAB_SIZE, dtype=int)
  for token in tokenizer.tokenize(tweet):
    stem=get_stem(token)
    idx=token_to_idx.get(stem, None)
    if idx is not None:
      vector[idx]=1
    else:
      if(print_unknown):
        print('Nepoznata rec: ', token)
  return vector
    

In [0]:
tw_idx=1
pos_tws=positive.values.tolist()
neg_tws=negative.values.tolist()
print("tweet: {}".format(pos_tws[tw_idx][0]))
print("tweet: {}".format(tweet_to_vector(pos_tws[tw_idx][0])[:10]))
print("tweet: {}".format(neg_tws[tw_idx][0]))
print("tweet: {}".format(tweet_to_vector(neg_tws[tw_idx][0])[:10]))

tweet: Да, все-таки он немного похож на него. Но мой мальчик все равно лучше:D
tweet: [1 0 0 0 0 1 0 0 0 0]
tweet: Коллеги сидят рубятся в Urban terror, а я из-за долбанной винды не могу :(
tweet: [1 1 1 0 1 0 1 0 0 0]


In [0]:
try:
  del train_vectors
  del test_vectors
except:
  print('Nije definisano')
  
train_size = round((len(negative) + len(positive))*(1-TEST_SIZE))
test_size = len(negative) + len(positive) - train_size

train_vectors = np.zeros((train_size, VOCAB_SIZE), dtype=np.int_)
test_vectors = np.zeros((test_size, VOCAB_SIZE), dtype=np.int_)

Nije definisano


In [0]:
tweets=[]
cnt=0
cnt2=0

train_pos_size = round(len(positive)*(1-TEST_SIZE))
train_neg_size = round(len(negative)*(1-TEST_SIZE))

for tweet in negative.values.tolist():
  #print(tweet[0])
  tweets.append(tweet[0])
  if cnt < train_neg_size:
    train_vectors[cnt]=tweet_to_vector(tweet[0])
    cnt+=1
  else:
    test_vectors[cnt2]=tweet_to_vector(tweet[0])
    cnt2+=1
for tweet in positive.values.tolist():
  tweets.append(tweet[0])
  if cnt < train_neg_size + train_pos_size:
    train_vectors[cnt]=tweet_to_vector(tweet[0])
    cnt+=1
  else:
    test_vectors[cnt2]=tweet_to_vector(tweet[0])
    cnt2+=1


In [0]:
print(len(train_vectors))
print(len(train_vectors[0]))
print(train_vectors)

226834
5000
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


##Brisanje vektora tvitova

In [0]:
del train_vectors
del test_vectors

##Labele

In [0]:
#labels = np.append(np.zeros(len(negative), dtype=np.int_), np.ones(len(positive), dtype=np.int_))

train_labels = np.append(
    np.zeros(train_neg_size, dtype=np.int_), 
    np.ones(train_pos_size, dtype=np.int_))

test_labels = np.append(
    np.zeros(len(negative) - train_neg_size, dtype=np.int_), 
    np.ones(len(positive) - train_pos_size, dtype=np.int_))

In [0]:
y_train = to_categorical(train_labels, 2)
y_test = to_categorical(test_labels, 2)


##Kreiranje modela

In [0]:
def build_model(learning_rate=0.1):
    tf.reset_default_graph()
    
    net = tflearn.input_data([None, VOCAB_SIZE])
    net = tflearn.fully_connected(net, 125, activation='ReLU')
    net = tflearn.fully_connected(net, 25, activation='ReLU')
    net = tflearn.fully_connected(net, 2, activation='softmax')
    regression = tflearn.regression(
        net, 
        optimizer='sgd', 
        learning_rate=learning_rate, 
        loss='categorical_crossentropy')
    
    model = tflearn.DNN(net)
    return model

In [0]:

model = build_model(learning_rate=0.75)

In [0]:
model.fit(
    X_train, 
    y_train, 
    validation_set=0.1, 
    show_metric=True, 
    batch_size=128, 
    n_epoch=30)